In [1]:
pip install flask flask-cors scikit-learn pandas numpy pymongo


Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the dataset
data = pd.read_csv("heart.csv")

# Features and target variable
X = data.drop(columns=['HeartDisease'])  # Remove the target column from features
y = data['HeartDisease']  # Target variable

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder # Import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer # Import ColumnTransformer

# Load dataset
data = pd.read_csv("heart.csv")

# Features and target variable
# Assuming 'HeartDisease' is the target variable and you want to drop it from features
X = data.drop(columns=['HeartDisease'])  # Changed to 'HeartDisease'
y = data['HeartDisease']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Identify categorical features (e.g., columns containing strings)
categorical_features = X.select_dtypes(include=['object']).columns

# Create a ColumnTransformer to handle numerical and categorical features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), X.select_dtypes(exclude=['object']).columns), # Apply StandardScaler to numerical features
        ('cat', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), categorical_features)  # Apply OneHotEncoder to categorical features
    ])

# Fit and transform the data using the preprocessor
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

# Train model
model = LogisticRegression()
model.fit(X_train, y_train)

# Save the model, scaler, and encoder
with open("heart_model.pkl", "wb") as f:
    pickle.dump(model, f)

with open("preprocessor.pkl", "wb") as f: # Save the preprocessor instead of just the scaler
    pickle.dump(preprocessor, f)

In [3]:
# Load trained model and preprocessor
with open("heart_model.pkl", "rb") as f:
    model = pickle.load(f)

with open("preprocessor.pkl", "rb") as f: # Changed from 'scaler.pkl' to 'preprocessor.pkl'
    preprocessor = pickle.load(f) # Changed 'scaler' to 'preprocessor'

In [6]:
# Load trained model and preprocessor
with open("heart_model.pkl", "rb") as f:
    model = pickle.load(f)

with open("preprocessor.pkl", "rb") as f: # Changed from 'scaler.pkl' to 'preprocessor.pkl'
    preprocessor = pickle.load(f) # Changed 'scaler' to 'preprocessor'

from flask import Flask, request, jsonify, render_template
import numpy as np

app = Flask(__name__)  # Initialize Flask app only once

@app.route('/predict', methods=['POST'])
def predict():
    try:
        data = request.json
        features = np.array(data["features"]).reshape(1, -1)
        # Use the preprocessor to transform the features
        features = preprocessor.transform(pd.DataFrame([data["features"]], columns=X.columns)) # Transform using preprocessor

        prediction = model.predict(features)[0]
        probability = model.predict_proba(features)[0][1]

        result = {
            "prediction": int(prediction),
            "probability": float(probability)
        }
        return jsonify(result)
    except Exception as e:
        return jsonify({"error": str(e)})

if __name__ == '__main__':
    app.run(debug=True) # Start Flask app when script is run

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (fsevents)
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/opt/anaconda3/lib/python3.12/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/opt/anaconda3/lib/python3.12/site-packages/traitlets/config/application.py", line 1074, in launch_instance
    app.initialize(argv)
  File "/opt/anaconda3/lib/python3.12/site-packages/traitlets/config/application.py", line 118, in inner
    return method(app, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  Fil

SystemExit: 1

/opt/anaconda3/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [7]:

1

1

In [8]:
!pip install pymongo


In [9]:
import pymongo
print(pymongo.__version__)  # Correct way to get version


4.11.2


In [10]:
!python -c "import pymongo; print(pymongo.__version__)"


4.11.2


In [11]:
import pymongo

# Connect to local MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")

# List available databases
print(client.list_database_names())


['admin', 'config', 'healthcare', 'local']


In [12]:
from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017/")
db = client["healthcare"]
doctors_collection = db["doctors"]

doctors = [
    {"doctor_id": "D123", "name": "Dr. John Doe", "specialization": "Cardiologist", "available_slots": ["10 AM", "2 PM"]},
    {"doctor_id": "D456", "name": "Dr. Jane Smith", "specialization": "Cardiologist", "available_slots": ["11 AM", "4 PM"]}
]

doctors_collection.insert_many(doctors)
print("Doctors inserted successfully!")


Doctors inserted successfully!


In [13]:
# Load trained model and preprocessor
with open("heart_model.pkl", "rb") as f:
    model = pickle.load(f)

with open("preprocessor.pkl", "rb") as f: # Changed from 'scaler.pkl' to 'preprocessor.pkl'
    preprocessor = pickle.load(f) # Changed 'scaler' to 'preprocessor'

from flask import Flask, request, jsonify, render_template
import numpy as np

app = Flask(__name__)  # Initialize Flask app only once

@app.route('/predict', methods=['POST'])
def predict():
    try:
        data = request.json
        features = np.array(data["features"]).reshape(1, -1)
        # Use the preprocessor to transform the features
        features = preprocessor.transform(pd.DataFrame([data["features"]], columns=X.columns)) # Transform using preprocessor

        prediction = model.predict(features)[0]
        probability = model.predict_proba(features)[0][1]

        result = {
            "prediction": int(prediction),
            "probability": float(probability)
        }
        return jsonify(result)
    except Exception as e:
        return jsonify({"error": str(e)})

if __name__ == '__main__':
    app.run(debug=True) # Start Flask app when script is run
    

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (fsevents)
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/opt/anaconda3/lib/python3.12/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/opt/anaconda3/lib/python3.12/site-packages/traitlets/config/application.py", line 1074, in launch_instance
    app.initialize(argv)
  File "/opt/anaconda3/lib/python3.12/site-packages/traitlets/config/application.py", line 118, in inner
    return method(app, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  Fil

SystemExit: 1

/opt/anaconda3/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
with open("heart_model.pkl", "rb") as f:
    model = pickle.load(f)

with open("preprocessor.pkl", "rb") as f: # Changed from 'scaler.pkl' to 'preprocessor.pkl'
    preprocessor = pickle.load(f) # Changed 'scaler' to 'preprocessor'

from flask import Flask, request, jsonify, render_template
import numpy as np
from flask import Flask
from threading import Thread

app = Flask(__name__)

@app.route("/")
def home():
    return "Hello, Flask in Jupyter Notebook!"


@app.route('/predict', methods=['POST'])
def predict():
    try:
        data = request.json
        features = np.array(data["features"]).reshape(1, -1)
        print(features)
        # Use the preprocessor to transform the features
        features = preprocessor.transform(pd.DataFrame([data["features"]], columns=X.columns)) # Transform using preprocessor
        print(features)
        prediction = model.predict(features)[0]
        probability = model.predict_proba(features)[0][1]

        result = {
            "prediction": int(prediction),
            "probability": float(probability)
        }
        return jsonify(result)
    except Exception as e:
        return jsonify({"error": str(e)})

def run():
    app.run(host="0.0.0.0", port=8081)

# Run Flask in a separate thread
thread = Thread(target=run)
thread.start()

 * Serving Flask app '__main__'


 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8081
 * Running on http://192.168.1.151:8081
Press CTRL+C to quit


In [15]:
X

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up
...,...,...,...,...,...,...,...,...,...,...,...
913,45,M,TA,110,264,0,Normal,132,N,1.2,Flat
914,68,M,ASY,144,193,1,Normal,141,N,3.4,Flat
915,57,M,ASY,130,131,0,Normal,115,Y,1.2,Flat
916,57,F,ATA,130,236,0,LVH,174,N,0.0,Flat


In [16]:
{
    "features": [63, 1, 3, 145, 233, 1, 0, 150, 0, 2.3, 0, 0, 1]
}


{'features': [63, 1, 3, 145, 233, 1, 0, 150, 0, 2.3, 0, 0, 1]}

In [17]:
{
    "prediction": 1,
    "probability": 0.85
}


{'prediction': 1, 'probability': 0.85}

In [18]:
!pip install requests


In [19]:
import requests

# Define the URL of your Flask API
url = "http://127.0.0.1:8082/predict"  # Localhost URL

# Example test data (modify based on your model's features)
test_data = {
    "features": [37,'M','ATA',130,283,0,'ST',98,'N',0,'Up']
}

# Send POST request to the API
response = requests.post(url, json=test_data)

# Print the response from the API
print(response.json())


ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=8082): Max retries exceeded with url: /predict (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x175ae31a0>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [ ]:
from flask import Flask, request, jsonify, render_template
import numpy as np
import pandas as pd
import pickle
from threading import Thread

# Load the trained model and preprocessor
with open("heart_model.pkl", "rb") as f:
    model = pickle.load(f)

with open("preprocessor.pkl", "rb") as f:
    preprocessor = pickle.load(f)

app = Flask(__name__)

# Route to render the HTML form
@app.route("/")
def home():
    return render_template("index.html")

# API endpoint to handle predictions
@app.route('/predict', methods=['POST'])
def predict():
    try:
        data = request.json
        raw_features = data["features"]

        # Convert numerical values from string to proper types
        processed_features = [
            int(raw_features[0]),  # Age
            raw_features[1],        # Gender (string, remains unchanged)
            raw_features[2],        # Chest Pain Type (string, remains unchanged)
            int(raw_features[3]),  # Resting BP
            int(raw_features[4]),  # Cholesterol
            int(raw_features[5]),  # Fasting BS (0 or 1)
            raw_features[6],        # Resting ECG (string, remains unchanged)
            int(raw_features[7]),  # Max HR
            raw_features[8],        # Exercise Angina (string, remains unchanged)
            float(raw_features[9]), # Oldpeak (decimal value)
            raw_features[10]        # ST Slope (string, remains unchanged)
        ]
        features=processed_features

        # Convert to DataFrame for transformation
        features_df = pd.DataFrame([processed_features], columns=X.columns)

        # Transform using the preprocessor
        transformed_features = preprocessor.transform(features_df)

        # Make prediction
        prediction = model.predict(transformed_features)[0]
        probability = model.predict_proba(transformed_features)[0][1]

        return jsonify({
            "prediction": int(prediction),
            "probability": float(probability)
        })
    except Exception as e:
        return jsonify({"error": str(e)})


# Function to run Flask server in a separate thread
def run():
    app.run(host="0.0.0.0", port=8082)

# Start Flask server
thread = Thread(target=run)
thread.start()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8082
 * Running on http://192.168.1.151:8082
Press CTRL+C to quit


In [21]:
import requests

# Define the URL of your Flask API
url = "http://127.0.0.1:8081/predict"  # Localhost URL

# Example test data (modify based on your model's features)
test_data = {
    "features": [37,'M','ATA',130,283,0,'ST',98,'N',0,'Up']
}

# Send POST request to the API
response = requests.post(url, json=test_data)

# Print the response from the API
print(response.json())


127.0.0.1 - - [10/Mar/2025 20:24:22] "POST /predict HTTP/1.1" 200 -


[['37' 'M' 'ATA' '130' '283' '0' 'ST' '98' 'N' '0' 'Up']]
[[-1.77937472 -0.16628515  0.77042425 -0.5427086  -1.50831556 -0.83628643
   0.          1.          0.          1.          0.          0.
   0.          0.          1.          1.          0.          0.
   0.          1.        ]]
{'prediction': 0, 'probability': 0.03517574684483676}


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the dataset
data = pd.read_csv("heart.csv")

# Features and target variable
X = data.drop(columns=['HeartDisease'])  # Remove the target column from feature